In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql.types import IntegerType
import  pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import desc
from pyspark.sql.functions import asc
#from pyspark.sql.functions import sum as Fsum


import datetime

import numpy as np
import pandas as pd

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
#Immig_data =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')
city_data=spark.read.csv('us-cities-demographics.csv',sep=";", inferSchema=True, header=True)

In [5]:
#Immig_data.write.mode('overwrite').parquet("sas_data")
Immmig_data=spark.read.parquet("sas_data")

In [6]:
Immmig_data.printSchema()

root
 |-- cicid: double (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- i94res: double (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- i94addr: string (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94bir: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- count: double (nullable = true)
 |-- dtadfile: string (nullable = true)
 |-- visapost: string (nullable = true)
 |-- occup: string (nullable = true)
 |-- entdepa: string (nullable = true)
 |-- entdepd: string (nullable = true)
 |-- entdepu: string (nullable = true)
 |-- matflag: string (nullable = true)
 |-- biryear: double (nullable = true)
 |-- dtaddto: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- insnum: string (nullable = true)
 |-- airline: string (nullable = true)
 |-- admnum: double (nullable = 

In [7]:
Immmig_data.show(5)

+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|    cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+---------+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     CA|20582.0|  40.0|    1.0|  1.0|20160430|     SYD| null|      G|      O|   null|      M| 1976.0|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518.0|2016.0|   4.0| 245.0| 438.0|    LOS|20574.0|    1.0|     NV|20591.0|  32.0|    1.0|  

In [ ]:
city_data.printSchema()

In [ ]:
city_data.show(5)

In [8]:
#Exploring the data and identifying the data issues.
Immmig_data.count()

3096313

In [62]:
#Based on the Cicid count which matches with Total cound of Immigration data we can consider cicid to unique with no duplictae values
ccid_count = State_dtls.select(countDistinct("cicid"))
ccid_count.show()

+--------------------------+
|count(DISTINCT state_code)|
+--------------------------+
|                        49|
+--------------------------+



In [ ]:
#checking below if we have any missing values for the required columns in Immigration data. 
Immmig_data.describe("i94addr","i94yr","i94mon","arrdate","visatype","i94mode").show()

In [12]:
#Dropping rows with null values
Immmig_data = Immmig_data.dropna(how = "any", subset = ["cicid", "i94addr","i94mode"])

In [12]:
#Dropped all the rows with no address as our analysis is only based on the state codes in US. 
Immmig_data.describe("cicid","i94addr","i94yr","i94mon","arrdate","visatype","i94mode").show()

+-------+------------------+------------------+-------+-------+------------------+--------+------------------+
|summary|             cicid|           i94addr|  i94yr| i94mon|           arrdate|visatype|           i94mode|
+-------+------------------+------------------+-------+-------+------------------+--------+------------------+
|  count|           2943721|           2943721|2943721|2943721|           2943721| 2943721|           2943721|
|   mean|3048991.7026433554|51.652482269503544| 2016.0|    4.0|20559.843950224902|    null|1.0541477945769997|
| stddev|1748336.5957907767| 42.97906231370983|    0.0|    0.0| 8.776730783982536|    null|0.4522191352762033|
|    min|               7.0|                ..| 2016.0|    4.0|           20545.0|      B1|               1.0|
|    max|         6099247.0|                ZU| 2016.0|    4.0|           20574.0|      WT|               9.0|
+-------+------------------+------------------+-------+-------+------------------+--------+------------------+



In [13]:
#Checking if there are any Empty Values in Cicid column
Immmig_data.select("cicid").dropDuplicates().sort("cicid").show()

+-----+
|cicid|
+-----+
|  7.0|
| 15.0|
| 16.0|
| 17.0|
| 18.0|
| 19.0|
| 20.0|
| 21.0|
| 22.0|
| 23.0|
| 24.0|
| 27.0|
| 28.0|
| 29.0|
| 30.0|
| 31.0|
| 33.0|
| 34.0|
| 35.0|
| 36.0|
+-----+
only showing top 20 rows



In [14]:
#Changing the data types from Double to Int for all the Numeric Fields.I could have done this by defining a new schema for \
#Immigration data and infering this schema when importing the data.
#Converting the SAS Numeric date form to actual date form.
Immmig_data = Immmig_data.withColumn('i94yr', expr("cast(i94yr  as int)")) \
.withColumn('i94mon', expr("cast(i94mon  as int)")) \
.withColumn('arrdate', expr("date_add('1960-01-01',arrdate)")) \
.withColumn('depdate', expr("date_add('1960-01-01',depdate)")) \
.withColumn('cicid', expr("cast(cicid  as int)")) \
.withColumn('i94mode', expr("cast(i94mode  as int)")) \
.withColumn('i94bir', expr("cast(i94bir  as int)")) \
.withColumn('i94visa', expr("cast(i94visa  as int)")) \
.withColumn('biryear', expr("cast(biryear  as int)"))

In [ ]:
Immmig_data.groupby("i94addr").count().orderBy(desc("count")).show()


In [38]:
Immmig_data.show(5)

+-------+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  cicid|i94yr|i94mon|i94cit|i94res|i94port|   arrdate|i94mode|i94addr|   depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-------+-----+------+------+------+-------+----------+-------+-------+----------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|5748517| 2016|     4| 245.0| 438.0|    LOS|2016-04-30|      1|     CA|2016-05-08|    40|      1|  1.0|20160430|     SYD| null|      G|      O|   null|      M|   1976|10292016|     F|  null|     QF|9.495387003E10|00011|      B1|
|5748518| 2016|     4| 245.0| 438.0|    LOS|2016-04-30|      1|     NV|2016-05-17|  

In [15]:
city_data.count()

2891

In [16]:
city_data.describe("City","State","Male Population","Female Population","State Code","Median Age").show()

+-------+-------+---------+------------------+------------------+----------+-----------------+
|summary|   City|    State|   Male Population| Female Population|State Code|       Median Age|
+-------+-------+---------+------------------+------------------+----------+-----------------+
|  count|   2891|     2891|              2888|              2888|      2891|             2891|
|   mean|   null|     null| 97328.42624653739|101769.63088642659|      null|35.49488066413016|
| stddev|   null|     null|216299.93692873296|231564.57257148277|      null|4.401616730099886|
|    min|Abilene|  Alabama|             29281|             27348|        AK|             22.9|
|    max|   Yuma|Wisconsin|           4081698|           4468707|        WI|             70.5|
+-------+-------+---------+------------------+------------------+----------+-----------------+



In [17]:
#Dropping Null rows if there are any.
city_data = city_data.dropna(how = "any", subset = ["City", "State","State Code"])

In [18]:
city_data.count()

2891

In [18]:
#validating why we have duplicate values for each city and State
city_data.select("City", "State","State Code","Total Population","Race","Count").filter(city_data["City"].isin(["Lawrence"])).sort("Count").show()

+--------+-------------+----------+----------------+--------------------+-----+
|    City|        State|State Code|Total Population|                Race|Count|
+--------+-------------+----------+----------------+--------------------+-----+
|Lawrence|Massachusetts|        MA|           80240|               Asian| 2149|
|Lawrence|       Kansas|        KS|           93916|American Indian a...| 4239|
|Lawrence|       Kansas|        KS|           93916|Black or African-...| 5777|
|Lawrence|       Kansas|        KS|           93916|  Hispanic or Latino| 6435|
|Lawrence|       Kansas|        KS|           93916|               Asian| 6547|
|Lawrence|Massachusetts|        MA|           80240|Black or African-...| 9221|
|Lawrence|Massachusetts|        MA|           80240|               White|40757|
|Lawrence|Massachusetts|        MA|           80240|  Hispanic or Latino|63175|
|Lawrence|       Kansas|        KS|           93916|               White|79793|
+--------+-------------+----------+-----

In [19]:
#changing the name of the columns to remove trailing whitespaces.
city_data = city_data.selectExpr("City as city","State as state","`Median Age` as Median_age",\
                                 "`Male Population` as Male_Population","`Female Population` as Female_Population",\
                                 "`Total Population` as Total_Population","`Number of Veterans` as Total_Veterans",\
                                 "`Foreign-born` as Foreign_born","`Average Household Size` as Average_Household_Size",\
                                 "`State Code` as State_Code","Race as race","count as count")

In [ ]:
city_data.show(5)

In [20]:
#Processing both immigration and City data to create analytics tables
#Creating Immigration view to store data in staging table
Immmig_data.createOrReplaceTempView("stg_Immig_data")
city_data.createOrReplaceTempView("stg_city_data")

In [44]:
# extract columns to create Immig_dtls table
Immig_dtls = spark.sql("""
             select Distinct 
             a.cicid as visitor_id,
             a.i94yr,
             a.i94mon,
             a.arrdate as arrival_date,
             Case when a.i94mode = 1 then 'AIR' \
             when a.i94mode = 2 then 'Sea'\
             when a.i94mode = 3 then 'Land' else 'other' end as i94mode,
             a.depdate as departure_date,
             a.i94bir as Age,
             Case when a.i94visa = 1 then 'Business'\
                  when a.i94visa = 2 then 'Pleasure' \
                  when a.i94visa = 3 then 'Student' else 'other' end as i94visa,
             a.dtadfile as create_dt,
             a.visapost ,
             a.biryear as birthyr,
             a.gender,
             a.airline,
             a.visatype
             from
             stg_Immig_data  a left join
             stg_city_data b on a.i94addr = b.State_Code
             where 
             a.cicid is not null 
          """)

In [ ]:
# write Immig_dtls table to parquet files partitioned by i94State. if more yrs of data is added,I would partition the data by year and state or year \
#or year and month.
Immig_dtls.write.mode('overwrite').partitionBy("i94State").parquet("Immigration_dtls")

In [67]:
Immig_dtls.createOrReplaceTempView("dim_Immig_dtls")
#city_data.createOrReplaceTempView("stg_city_data")

In [30]:
Immig_dtls.count()

2943721

In [59]:
# extract columns to create State_dtls table
State_dtls = spark.sql("""
             select Distinct
             monotonically_increasing_id() as state_id,
             a.state as state,
             a.State_Code,
             a.city
             from
             stg_city_data  a 
             where
             a.State_Code is not null
          """)

In [66]:
# write users table to parquet files
#State_dtls.write.mode('overwrite').parquet("State Details")
State_dtls.createOrReplaceTempView("dim_State_dtls")

In [60]:
spark.sql("""
             select Distinct
             monotonically_increasing_id() as state_id,
             a.state as state,
             a.State_Code,
             a.city
             from
             stg_city_data  a 
             where
             a.State_Code is not null
          """)

+--------+----------+----------+--------------+
|state_id|     state|State_Code|          city|
+--------+----------+----------+--------------+
|     245|     Texas|        TX|       Abilene|
|     352|  Colorado|        CO|  Fort Collins|
|     583|   Georgia|        GA|      Savannah|
|     630|   Arizona|        AZ|San Tan Valley|
|     886|California|        CA|        Rialto|
+--------+----------+----------+--------------+
only showing top 5 rows



In [64]:
# extract columns to create Date_table 
Date_table = spark.sql("""
             select Distinct
             a.arrdate as Date,
             dayofmonth(a.arrdate) as day,
             weekofyear(a.arrdate) as week,
             month(a.arrdate) as month,
             year(a.arrdate) as year,
             dayofweek(a.arrdate) as weekday
             from
             stg_Immig_data  a 
             where
             a.arrdate is not null
          """)

In [68]:
Date_table.createOrReplaceTempView("dim_Date_table")

In [75]:
# write data table to parquet files partitioned by year and month
#Date_table.write.mode('overwrite').partitionBy("year", "month").parquet("Date_Table")
Date_table.show(5)

+----------+---+----+-----+----+-------+
|      Date|day|week|month|year|weekday|
+----------+---+----+-----+----+-------+
|2016-04-07|  7|  14|    4|2016|      5|
|2016-04-30| 30|  17|    4|2016|      7|
|2016-04-05|  5|  14|    4|2016|      3|
|2016-04-14| 14|  15|    4|2016|      5|
|2016-04-22| 22|  16|    4|2016|      6|
+----------+---+----+-----+----+-------+
only showing top 5 rows



In [50]:
# extract columns to create Immig_visit fact table
Immig_visit = spark.sql("""
             select Distinct 
             monotonically_increasing_id() as Immig_visit_id,
             a.cicid as visitor_id,
             a.arrdate as Date,
             month(a.arrdate) as month,
             year(a.arrdate) as year,
             a.i94mode,
             a.i94visa,
             a.matflag as Match_flag,
             Case when a.i94addr = b.State_Code then a.i94addr \
             else '99' end as i94State
             from
             stg_Immig_data  a left join
             stg_city_data b on a.i94addr = b.State_Code
             where 
             a.cicid is not null 
          """)

In [69]:
# write Immig_visit table to parquet files partitioned by i94addr
#Immig_visit.write.mode('overwrite').partitionBy("i94addr").parquet("Immig_Visit")
Immig_visit.createOrReplaceTempView("Fact_Immig_visit")


In [ ]:
Immig_visit.printSchema()

In [73]:
spark.sql("""
             select
             count(Distinct a.visitor_id),
             b.i94mode,
             c.week,
             d.state
             from
             Fact_Immig_visit  a inner join 
             dim_Immig_dtls b on a.visitor_id = b.visitor_id inner join
             dim_Date_table c on a.date = c.date inner join 
             dim_State_dtls d on d.state_code = i94State
             where
             a.i94mode = 1
             group by 
             a.i94mode,c.week,d.state
          """).show(10)

+--------------------------+-------+
|count(DISTINCT visitor_id)|i94mode|
+--------------------------+-------+
|                   2877095|      1|
|                     48488|      3|
|                      6326|      9|
+--------------------------+-------+
only showing top 3 rows

